In [ ]:
!pip install pyngrok

In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install --upgrade yolov5

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
!pip install flask flask_cors

In [ ]:
from flask import Flask, request, jsonify
import torch
import cv2
import numpy as np
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.general import non_max_suppression, scale_boxes
from yolov5.utils.augmentations import letterbox
from flask_cors import CORS
import base64
from pyngrok import ngrok  # Import pyngrok
import os

# Define pest class names
PEST_NAMES = [
    "Aphid", "Armyworm", "Bollworm", "Cutworm", "Grasshopper", "Leafhopper",
    "Locust", "Mite", "Thrips", "Weevil"
]

app = Flask(__name__)
CORS(app)

# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2slaSnqKE9lgdbHHad77CdYVDZ4_3KL8Ti6kRJnJg63xujnUa"  # Get token from environment variable
# Or you can directly set it here:
# NGROK_AUTH_TOKEN = "YOUR_AUTH_TOKEN"  # Replace with your actual authtoken

# Configure ngrok with your authtoken
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start ngrok tunnel
public_url = ngrok.connect(5009)
print("Public URL:", public_url)


# Load YOLOv5 model
def load_model(model_path):
    model = DetectMultiBackend(model_path, device='cpu')
    model.eval()
    return model

# Preprocess image for YOLOv5
def preprocess_image(image, img_size=640):
    img = cv2.imdecode(np.frombuffer(image, np.uint8), cv2.IMREAD_COLOR)
    img = letterbox(img, img_size, stride=32, auto=True)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)
    img = np.ascontiguousarray(img, dtype=np.float32) / 255.0
    return torch.tensor(img).unsqueeze(0)

# Run inference
def run_inference(model, image_tensor, conf_thresh=0.4, iou_thresh=0.5):
    with torch.no_grad():
        preds = model(image_tensor)
    preds = non_max_suppression(preds, conf_thresh, iou_thresh)
    return preds

# Draw bounding boxes on image
def draw_boxes(image, preds, image_tensor):
    img = cv2.imdecode(np.frombuffer(image, np.uint8), cv2.IMREAD_COLOR)
    detected_pests = []
    for det in preds:
        if det is not None and len(det):
            det[:, :4] = scale_boxes(image_tensor.shape[2:], det[:, :4], img.shape).round()
            for *xyxy, conf, cls in det:
                class_idx = int(cls)
                pest_name = PEST_NAMES[class_idx] if class_idx < len(PEST_NAMES) else f"Unknown {class_idx}"
                detected_pests.append(pest_name)

                label = f"{pest_name}: {conf:.2f}"
                cv2.rectangle(img, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (0, 255, 0), 2)
                cv2.putText(img, label, (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    _, img_encoded = cv2.imencode('.jpg', img)
    img_base64 = base64.b64encode(img_encoded).decode('utf-8')  # Convert image to base64
    return img_base64, detected_pests

# Load the trained model
model_path = "best.pt"  # Update with your trained model path
model = load_model(model_path)

@app.route('/')
def home():
    return "Pest Detection API Running!"

@app.route('/detect', methods=['POST'])
def detect_pests():
    if 'file' not in request.files:
        return jsonify({"error": "No file provided"}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    try:
        image_bytes = file.read()
        image_tensor = preprocess_image(image_bytes)
        preds = run_inference(model, image_tensor)
        output_image_base64, detected_pests = draw_boxes(image_bytes, preds, image_tensor)

        return jsonify({
            "message": "Detection successful",
            "detected_pests": detected_pests,
            "output_image": output_image_base64  # Sending image as base64
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(port=5009)


Public URL: NgrokTunnel: "https://2adc-35-234-29-5.ngrok-free.app" -> "http://localhost:5009"


Fusing layers... 
Model summary: 214 layers, 7035811 parameters, 0 gradients, 16.0 GFLOPs


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5009
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Feb/2025 13:10:05] "OPTIONS /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Feb/2025 13:10:07] "POST /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Feb/2025 13:12:42] "OPTIONS /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Feb/2025 13:12:43] "POST /detect HTTP/1.1" 200 -


In [ ]:
!pkill -f ngrok
